Туториал https://towardsdatascience.com/word2vec-from-scratch-with-numpy-8786ddd49e72

# Нормализуем текст, чтобы его лекго было разбить на слова

In [74]:
import string
import numpy as np

text = 'After the deduction of the costs of investing, beating the stock market is a loser’s game.'

def preprocess_text(s):
    return s.lower().translate(str.maketrans('', '', string.punctuation))

text = preprocess_text(text)
text

'after the deduction of the costs of investing beating the stock market is a loser’s game'

# Генерируем обучающие данные

In [43]:
def is_window_out_of_range(text, window_center, window_size):
    return window_center < window_size or window_center + window_size >= len(text)

def get_window(text, window_center, window_size):
    if is_window_out_of_range(text, window_center, window_size):
        raise ValueError('window out of range')
    return [(text[window_center], text[i]) for i in range(window_center-window_size, window_center+window_size+1) if i!=window_center]

def generate_training_data(text, window_size):
    return [get_window(text, i, window_size) for i in range(len(text)) if not is_window_out_of_range(text, i, window_size)]
    
training_data = generate_training_data(text.split(), 5)
training_data[0]

[('costs', 'after'),
 ('costs', 'the'),
 ('costs', 'deduction'),
 ('costs', 'of'),
 ('costs', 'the'),
 ('costs', 'of'),
 ('costs', 'investing'),
 ('costs', 'beating'),
 ('costs', 'the'),
 ('costs', 'stock')]

# Инициализируем модель

In [56]:
vocabulary = list(set(text.split()))
vocab_size = len(vocabulary)
emb_size = 3

word_emb = np.random.rand(vocab_size, emb_size)
dense = np.random.rand(vocab_size, emb_size)
dense

array([[0.69388698, 0.73742935, 0.69798044],
       [0.18304424, 0.37327225, 0.69995864],
       [0.83417978, 0.12099023, 0.63048464],
       [0.53263056, 0.30788734, 0.15992509],
       [0.99710519, 0.07411334, 0.11812715],
       [0.71773656, 0.41104157, 0.09350483],
       [0.56515052, 0.24317991, 0.33192345],
       [0.03015776, 0.56734375, 0.15828918],
       [0.08361993, 0.04738956, 0.70153531],
       [0.23606822, 0.74651537, 0.96474563],
       [0.02305037, 0.29269734, 0.75891743],
       [0.89448726, 0.08299992, 0.61331541],
       [0.58317863, 0.90167942, 0.56439279]])

# Forward propagation

In [92]:
def forward(word):
    emb = np.array(word_emb[vocabulary.index(word)]).reshape(-1,1)
    res = np.dot(dense, emb)
    res = np.exp(res)
    res = res / res.sum()
    return res

def interpretate(vec):
    vec = [(i, x) for i, [x] in enumerate(vec)]
    vec.sort(key=lambda x: x[1], reverse=True)
    return [(vocabulary[i], p) for i, p in vec]

def loss(vec):
    a = np.log(vec.mean())
    vec = vec * a
    return -1/vec.shape[0]*vec.sum()

interpretate(forward('the'))
loss(forward('the'))

0.19730379672781054

# Backward propagation